<a href="https://colab.research.google.com/github/SunsetSage/purity-ui-dashboard/blob/main/prompt.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install openai
!pip install tenacity
!pip install rich
from openai import OpenAI

In [ ]:


# this template will be used on the first run to:
# - identify any changes
# - flag the number of changes on a node
FIRST_REVIEW = """
```Role```

You are a content review agent tasked with reviewing content from a tiptap document.
Your role is to review the content in light of specific restrictions, identifying any discrepancies
 and state output the node ID and the reason for the change.

```Input Format```

The input is a tiptap document in JSON format.

```Output Format```

 The output must only include a list of nodes to be CHANGED nodes in valid JSON format."
        A node must not be included in the output if it is not changed.
        - The id must be of the text node, not the parent paragraph.
        - The reason must specify why id does not match the requirements

        Example output if there are changes:
        \n
        [ {{
           "id": <the id of the text node>,
            "reason": <short reason for the change>,
           "1": <number of changes required to node>
          }}
       ]

        Example output if there are no changes at all in the document.:
        \n
        "No changes"

    ```Restrictions```:
{restrictions}
       """


# instruct the agent to add the full updated text node.

SECOND_REVIEW = """
      ```Role```

You are a content review agent that is part of a chain of agents examining
content from a tiptap document.

Your role is to:
- review the content in light of specific restrictions,
- check if previously identified CHANGES are accurate.
Keep in mind that the previous content is reviewed by an equally capable AI.
- Do not make up rules that are not in the restrictions
 - use previously identified discrepancies to suggest changes to the content
 - state the reason for the change.

```Input Format```

- The input is a tiptap document in json format.
- The previous output in the review process.

```Output format Format```

 The output must include a list of ALL the CHANGED nodes in valid json format."
        Example output if there are changes:
        \n
        [ {{
           "id": <the id of the text node>,
            "text":<the updated text node with all of the changes.>,
            "reason": <the reason for the change>,
          }}
       ]

```Restrictions```
{restrictions}
"""



In [ ]:
from tenacity import retry, stop_after_attempt, wait_fixed
import json


@retry(stop=stop_after_attempt(3), wait=wait_fixed(1))
def open_ai_generate_first_pass(content, restrictions, prompt=None):
    """
    First pass generation.
    Override the prompt by passing in a new one.

    # EXAMPLE
    result = open_ai_generate_first_pass(
      [...content],
      [..restrictions],
      prompt=YOUR_CUSTOM_PROMPT
    )

    """

    client = OpenAI(api_key=OPENAI_API_KEY)
    prompt = prompt or FIRST_REVIEW

    # make sure the prompt includes `{restrictions}` in single curly brackets
    template = prompt.format(restrictions=restrictions)
    messages = [
            {
                "role": "system",
                "content": json.dumps(template)
            },
            {
                "role": "user",
                "content": json.dumps(content)
            },
        ]

    response = client.chat.completions.create(
        model="gpt-4-1106-preview",
        messages=messages,
        temperature=0,
        max_tokens=4000,
        top_p=1,
        frequency_penalty=0,
        presence_penalty=0
    )
    message = response.choices[0].message.content


    return message

@retry(stop=stop_after_attempt(3), wait=wait_fixed(1))
def open_ai_generate(content, restrictions, prompt=None):
    """Second pass generation."""
    client = OpenAI(api_key=OPENAI_API_KEY)
    prompt = prompt or SECOND_REVIEW
    template = prompt.format(restrictions=restrictions)
    messages = [
            {
                "role": "system",
                "content": json.dumps(template)
            },
            {
                "role": "user",
                "content": json.dumps(content)
            },
        ]


    response = client.chat.completions.create(
        model="gpt-4-1106-preview",
        messages=messages,
        temperature=0,
        max_tokens=4000,
        top_p=1,
        frequency_penalty=0,
        presence_penalty=0
    )
    message = response.choices[0].message.content

    return message

In [ ]:
content = [{
  "type": "doc",
  "content": [
    {
      "type": "paragraph",
      "attrs": {
        "id": "3b31a5f5-8d6b-4d9e-a115-687f6558f3df"
      },
      "content": [
        {
          "text": "Hey there, Plant-Powered Palate Pleasers!",
          "type": "text"
        }
      ]
    },
    {
      "type": "paragraph",
      "attrs": {
        "id": "9a30b202-cb53-4fa5-8817-9bcfc43fd1e5"
      },
      "content": [
        {
          "text": "Are you ready to indulge without the guilt? We know you've been dreaming of a world where junk food doesn't mean junk values. Well, dream no more! Our Vegan Junk Food is here to satisfy your cravings and save our furry friends, all in one delicious bite.",
          "type": "text"
        }
      ]
    },
    {
      "type": "paragraph",
      "attrs": {
        "id": "4df0de40-6163-4bfe-8685-6d1194efb757"
      },
      "content": [
        {
          "text": "Now, let's talk about the elephant in the room – or rather, the animals NOT in the room. Our snacks are 100% plant-based, ensuring that no animals were harmed in the making of your munchies. It's a win-win for your taste buds and our friends buddies.",
          "type": "text"
        }
      ]
    },
    {
      "type": "paragraph",
      "attrs": {
        "id": "52677fed-7e38-486a-8e3b-640dbab95008"
      },
      "content": [
        {
          "text": "**Craving a taste? Click here to grab your first bag of guilt-free bliss!**",
          "type": "text"
        }
      ]
    },
    {
      "type": "paragraph",
      "attrs": {
        "id": "86e966fc-7b01-43a6-af81-52e3fb9a4347"
      },
      "content": [
        {
          "text": "Dive into our mouth-watering selection of veganized classics. From ooey-gooey cheese-flavored nachos to chocolatey, cream-filled cookies that could fool any dairy devotee, we've got your back when the snack attack hits. And the best part? You can crush those bags of chips and cookies knowing you're part of a movement that's kinder to beings.",
          "type": "text"
        }
      ]
    },
    {
      "type": "paragraph",
      "attrs": {
        "id": "8bd8e825-df01-45eb-ae12-b0faebefbb6f"
      },
      "content": [
        {
          "text": "Our snacks aren't just about being kind; they're about being bold. We're pushing the boundaries of what vegan can be, one snack at a time. So whether you're a full-time vegan or a curious carnivore, our flavors are designed to delight and surprise. Because who said vegan has to be vanilla?",
          "type": "text"
        }
      ]
    },
    {
      "type": "paragraph",
      "attrs": {
        "id": "9325b13e-d75f-4843-9907-0629cd1097e7"
      },
      "content": [
        {
          "text": "So, what are you waiting for? It's time to join the snacking revolution that's kind to your body, kind to the planet, and kind to animals. Let's make your snack drawer a testament to your values, without sacrificing an ounce of flavor.",
          "type": "text"
        }
      ]
    },
    {
      "type": "paragraph",
      "attrs": {
        "id": "c70b9b59-1f13-43f2-9216-f1becf52aa47"
      },
      "content": [
        {
          "text": "**Ready to snack on the wild side? Click here to explore our range and get your crunch on!**",
          "type": "text"
        }
      ]
    },
    {
      "type": "paragraph",
      "attrs": {
        "id": "c7a3793c-4eb1-419e-a771-5718a7f06160"
      },
      "content": [
        {
          "text": "Remember, every bite is a stand against animal cruelty and a step towards a more sustainable future. So go ahead, treat yourself to the ultimate vegan junk food experience. Your taste buds, and the animals, will thank you.",
          "type": "text"
        }
      ]
    },
    {
      "type": "paragraph",
      "attrs": {
        "id": "9ea9c841-9793-4b67-8faa-2811b728dd4d"
      },
      "content": [
        {
          "text": "Stay kind, stay hungry, and most importantly, stay snacking!",
          "type": "text"
        }
      ]
    },
    {
      "type": "paragraph",
      "attrs": {
        "id": "00e53c3c-8153-4a7e-95ff-bd0c90cccf81"
      },
      "content": [
        {
          "text": "With kindness and crunch,\nYour Vegan Junk Food Heroes\n\nWe are the best in the world at this",
          "type": "text"
        }
      ]
    }
  ]
}]



In [ ]:
# RUN THE FIRST REVIEW
# -
import rich

restrictions = """
- Do not include the word kindness
- Do not include the word junk food.
"""

first_response = open_ai_generate_first_pass(
    content,
    restrictions,
    prompt=FIRST_REVIEW
)
rich.print(first_response)

```json
[
  {
    "id": "9a30b202-cb53-4fa5-8817-9bcfc43fd1e5",
    "reason": "Contains the word 'junk food'",
    "1": 1
  },
  {
    "id": "86e966fc-7b01-43a6-af81-52e3fb9a4347",
    "reason": "Contains the word 'kinder'",
    "1": 1
  },
  {
    "id": "8bd8e825-df01-45eb-ae12-b0faebefbb6f",
    "reason": "Contains the word 'kind'",
    "1": 1
  },
  {
    "id": "9325b13e-d75f-4843-9907-0629cd1097e7",
    "reason": "Contains the word 'kind' multiple times",
    "1": 2
  },
  {
    "id": "9ea9c841-9793-4b67-8faa-2811b728dd4d",
    "reason": "Contains the word 'kind'",
    "1": 1
  },
  {
    "id": "00e53c3c-8153-4a7e-95ff-bd0c90cccf81",
    "reason": "Contains the word 'kindness'",
    "1": 1
  }
]
```

In [ ]:
# RUN THE SECOND REVIEW

second_review_response = open_ai_generate(
    first_response, restrictions
  )
rich.print(second_review_response)

```json
[
  {
    "id": "9a30b202-cb53-4fa5-8817-9bcfc43fd1e5",
    "text": "<the updated text node without the word 'junk food'>",
    "reason": "Contains the word 'junk food'"
  },
  {
    "id": "00e53c3c-8153-4a7e-95ff-bd0c90cccf81",
    "text": "<the updated text node without the word 'kindness'>",
    "reason": "Contains the word 'kindness'"
  }
]
```
The other identified words 'kinder' and 'kind' are not restricted based on the given restrictions. Only 'junk food'
and 'kindness' are to be removed.